# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
# Your code here; import the necessary packages
import pandas as pd 
import sqlite3



In [2]:
# Your code here; create the connection
conn = sqlite3.connect("data.sqlite")

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [3]:
# Your code here
q  = """ 
 SELECT customerNumber,
      contactLastName,
      contactFirstName
 FROM customers
 WHERE customerNumber IN (SELECT customerNumber 
                          FROM orders
                          WHERE orderDate = '2003-01-31'
                      );
 """
sub_q = pd.read_sql(q, conn)
sub_q

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
# Your code here
q = """ 
    SELECT productName, 
                (SELECT COUNT(orderNumber)
                FROM orderDetails
                WHERE orderDetails.productCode = products.productCode) AS total_number_orders, 
                (SELECT SUM(quantityOrdered)
                FROM orderDetails
                WHERE orderDetails.productCode = products.productCode) AS total_units_sold
    FROM products
    GROUP BY productName
    ORDER BY total_units_sold DESC;
"""
pd.read_sql(q, conn)

,productName,total_number_orders,total_units_sold
0,1992 Ferrari 360 Spider red,53,1808.0
1,1937 Lincoln Berline,28,1111.0
2,American Airlines: MD-11S,28,1085.0
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076.0
4,1930 Buick Marquette Phaeton,28,1074.0
...,...,...,...
105,1911 Ford Town Car,25,832.0
106,1936 Mercedes Benz 500k Roadster,25,824.0
107,1970 Chevy Chevelle SS 454,25,803.0
108,1957 Ford Thunderbird,24,767.0


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [5]:
# Your code here
q = """ 
SELECT productName, 
            (SELECT COUNT(customerNumber)
            FROM orders
            WHERE orderNumber IN ( SELECT orderNumber
                                   FROM orderdetails
                                   WHERE orderdetails.productCode = products.productCode)) AS total_num_customers 
FROM products
GROUP BY productName
ORDER BY total_num_customers DESC;
"""
pd.read_sql(q, conn)

,productName,total_num_customers
0,1992 Ferrari 360 Spider red,53
1,P-51-D Mustang,28
2,HMS Bounty,28
3,F/A 18 Hornet 1/72,28
4,Diamond T620 Semi-Skirted Tanker,28
...,...,...
105,1917 Grand Touring Sedan,25
106,1911 Ford Town Car,25
107,1957 Ford Thunderbird,24
108,1952 Citroen-15CV,24


### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [51]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
q = """ 
SELECT DISTINCT customerName, customerNumber
          FROM customers
     LEFT JOIN orders
               USING(customerNumber)
         
        
"""

pd.read_sql(q, conn)

,customerName,customerNumber
0,Atelier graphique,103
1,Signal Gift Stores,112
2,"Australian Collectors, Co.",114
3,La Rochelle Gifts,119
4,Baane Mini Imports,121
...,...,...
117,Motor Mint Distributors Inc.,486
118,Signal Collectibles Ltd.,487
119,"Double Decker Gift Stores, Ltd",489
120,Diecast Collectables,495


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [47]:
q = """ 
    SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, o.city, e.officeCode
    FROM employees e
    LEFT JOIN offices o 
         USING(officeCode)
    LEFT JOIN customers c 
         ON e.employeeNumber = c.salesRepEmployeeNumber
    LEFT JOIN orders o 
         USING(customerNumber) 
    WHERE o.orderNumber IN (SELECT od.orderNumber
                              FROM orderdetails od
                              GROUP BY productCode
                              HAVING COUNT(od.orderNumber) < 20
);
    """
pd.read_sql(q, conn)

,employeeNumber,firstName,lastName,city,officeCode


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [31]:
q = """ 
  SELECT e.employeeNumber, e.firstName, e.lastName,
          COUNT(c.customerNumber) AS num_of_customers
    FROM employees AS e
    JOIN customers AS c 
         ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY e.employeeNumber, e.firstName, e.lastNam
    HAVING AVG(c.creditLimit) > 15000
"""
pd.read_sql(q, conn)

,employeeNumber,firstName,lastName,num_of_customers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!